# 若干小工具

## 将剪贴板内容去掉\n，去掉中文之间空格

In [ ]:
import pyperclip, re
txtsrc = pyperclip.paste()
reReturn = re.compile(r'\r|\n|\r\n')
reSpcBetwnChnChara = re.compile(r'([\u4e00-\u9fa5])\s+([\u4e00-\u9fa5])')
txtsrc = re.sub(reReturn,'',txtsrc)
txtsrc = re.sub(reSpcBetwnChnChara,r'\1\2',txtsrc)

dict_key = r"１２３４５６７８９０ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ，．／！＠＃＄％＾＆＊（）＜＞？；＇：｛｝＿＋－＝"
dict_val = r"1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ,./!@#$%^&*()<>？;':{}_+-="
dict_ = {}
reQuanjiao = re.compile(r'[{0}]'.format(dict_key))
for i in range(len(dict_key)):
    dict_[dict_key[i]]=dict_val[i]
dict_['＂']='"'

def quanjiao2banjiao(mo):
    c = mo.group(0)
    return dict_[c]

txtsrc = re.sub(reQuanjiao,quanjiao2banjiao,txtsrc)
print(txtsrc)
pyperclip.copy(txtsrc)

## manual sequencer

In [ ]:
import pyperclip,re
txt = pyperclip.paste().strip()
delim = ""
if '\r' in txt:
    delim += '\r'
if '\n' in txt:
    delim += '\n'
print('delim:%r'%delim)
startNum = 1
tlist = txt.split(delim)
for i in range(len(tlist)):
    tlist[i] = startNum+i
tlist = list(map(lambda e:str(e),tlist))
rst = delim.join(tlist)
print(rst)
pyperclip.copy(rst)

## gene sequence converter

In [ ]:
import pyperclip
class converter:
    dict_ = {
    'a':'t',
    't':'a',
    'c':'g',
    'g':'c',
    }
    def __init__(self,seq):
        self.seq = seq.lower()
        self.seqlist = list(self.seq)
    def complement(self):
        self.seqlist = list(map(lambda e:self.dict_[e],self.seqlist))
        return self
    def dna2rna(self):
        self.seqlist = list(map(lambda e:e.replace('t','u'),self.seqlist))
        return self
    def rna2dna(self):
        self.seqlist = list(map(lambda e:e.replace('u','t'),self.seqlist))
        return self
    def reverse(self):
        self.seqlist.reverse()
        return self
    def getSeq(self):
        return ''.join(self.seqlist)
    
seqori = pyperclip.paste()  
seq1 = converter(seqori)
rst = seq1.rna2dna().complement().dna2rna().reverse().getSeq()
print(rst)
pyperclip.copy(rst)

## 文库文档下载

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from selenium.webdriver.support.ui import WebDriverWait
import time
import pyperclip
browser = webdriver.Chrome()
browser.get('https://wenku.baidu.com/view/6b5d50c608a1284ac850438e.html')
# first finishe fire up the driver and load the page

In [ ]:
moreBtn = browser.find_element_by_class_name('moreBtn')
moreBtn.click()
pageInput = browser.find_element_by_class_name('page-input')
datalist = []
reTopVal = re.compile(r'top: (\d+)px')
def contentExtract(pageNum):
    pageInput.clear()
    pageInput.send_keys(str(pageNum))
    pageInput.send_keys(u'\ue007')
    pageElemId = "pageNo-"+str(pageNum)
    print(pageElemId)
    time.sleep(1)
    elem = browser.find_element_by_id(pageElemId)
    subelems = elem.find_elements_by_class_name("reader-word-layer")
    def getYpos(e):
        """获取一个字符block的style里面的top属性，相关的regex在函数外面已经compile完成"""
        mo = reTopVal.search(e.get_attribute('style'))
        return mo.group(1)
    def lineMerging(elems):
        """根据位置top信息判断是否属于一行，如果是新的一行加上换行符以后再连接文字"""
        topTemp = ""
        rstString = ""
        for e in elems:
            if topTemp == getYpos(e):
                rstString += e.text
            else:
                topTemp = getYpos(e)
                rstString += '\r\n' + e.text
        return rstString
    return lineMerging(subelems)
pageTtl = int(browser.find_element_by_class_name('page-count').text[1:])
# 这里根据最大页码手动填写一下范围。如果最大页码在50页以内就可以直接用pageTtl，但是如果超过了50页，这里要分两块才行
for i in range(pageTtl):
    print(i+1)
    datalist.append(contentExtract(i+1))
pyperclip.copy('\r\n'.join(datalist))

## 在文件夹名字前面加上拼音首字母前缀


In [ ]:
from pypinyin import lazy_pinyin
import os,shutil,re,pyperclip
pathofcwd = pyperclip.paste()
assert os.path.isdir(pathofcwd),'"{}" is not path'.format(pathofcwd)
os.chdir(pyperclip.paste())
reChnFirst = re.compile(r'[\u4e00-\u9fa5]')
dirList = list(filter(lambda e:os.path.isdir(e) and reChnFirst.match(e.strip()) != None,os.listdir()))
  
def renameDirWithPY (dirname):
    prefix = ''
    for e in lazy_pinyin(dirname)[:3]:
        prefix += e[0]
    prefix += '_'
    return prefix+dirname

for e in dirList:
    shutil.move(e,renameDirWithPY(e))
 
os.listdir()